### Классификация текстов при помощи правил

In [5]:
pip install yargy

Note: you may need to restart the kernel to use updated packages.


#### Правило для извлечения дат в ISO-формате («2018-02-23», «2015-12-31»)

In [1]:
from yargy import rule, and_, Parser
from yargy.predicates import gte, lte


DAY = and_(
    gte(1),
    lte(31)
)
MONTH = and_(
    gte(1),
    lte(12)
)
YEAR = and_(
    gte(1),
    lte(2022)
)
DATE = rule(
    YEAR,
    '-',
    MONTH,
    '-',
    DAY
)

parser = Parser(DATE)
text = '''
2018-02-23,
2015-12-31;
8 916 364-12-01'''
for match in parser.findall(text):
    print(match.span, [_.value for _ in match.tokens])

[1, 11) ['2018', '-', '02', '-', '23']
[13, 23) ['2015', '-', '12', '-', '31']
[31, 40) ['364', '-', '12', '-', '01']


#### Корректировка правила

In [6]:
from yargy import rule, and_, Parser
from yargy.predicates import gte, lte


DAY = and_(
    gte(1),
    lte(31)
)
MONTH = and_(
    gte(1),
    lte(12)
)
YEAR = and_(
    gte(2000),
    lte(2022)
)
DATE = rule(
    YEAR,
    '-',
    MONTH,
    '-',
    DAY
)

parser = Parser(DATE)
text = '''
2018-02-23,
2015-12-31;
8 916 364-12-01'''
for match in parser.findall(text):
    print(match.span, [_.value for _ in match.tokens])

[1, 11) ['2018', '-', '02', '-', '23']
[13, 23) ['2015', '-', '12', '-', '31']


#### Правила для дат вида «8 января 2014», «15 июня 2001 г.»

In [11]:
from yargy import or_
from yargy.predicates import caseless, normalized, dictionary


MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'мая',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
}
MONTH_NAME = dictionary(MONTHS)
YEAR_WORDS = or_(
    rule(caseless('г'), '.'),
    rule(normalized('год'))
)
DATE = or_(
    rule(
        YEAR,
        '-',
        MONTH,
        '-',
        DAY
    ),
    rule(
        DAY,
        MONTH_NAME,
        YEAR,
        YEAR_WORDS.optional()
    )
)

parser = Parser(DATE)
text = '''
8 января 2014 года, 15 июня 2001 г.,
31 февраля 2018'''
for match in parser.findall(text):
    print(match.span, [_.value for _ in match.tokens])

[1, 19) ['8', 'января', '2014', 'года']
[21, 36) ['15', 'июня', '2001', 'г', '.']
[38, 53) ['31', 'февраля', '2018']
